In [11]:
import collections
import re

### Byte-Pair Encoding

This is an algorithm for subword tokenization. It has two parts, a `token learner` and a `token segmenter`. The learner generates a vocabulary of subword type and the segmenter converts a test sentence into a sequence of subword tokens using the learned vocabulary.

To learn the vocabulary, we perform the following steps:

1) Given the input corpus of text (which is a single string), split across all whitespaces to get a set of strings. Then append a special end-of-word character "_" to each string. We define each of these strings to be a "word".
2) Count the frequency of all the unique words. Split each word into a list of its individual characters, which are the tokens.
3) Initialize the vocabulary as the set of all unique characters across these words, including the "_" character.
4) Find the pair of adjacent tokens that appear most frequently (in case of ties, just break in some consistent way). Merge the pair of tokens into a single new token, add this new token to the vocabulary and replace all occuracnces of this pair across all words with this new token.
5) Repeat step 4 until the vocabulary reaches some specified size.   

In addition to the learned vocabulary of all merged tokens, we also keep track of all the pairs that have been merged. Then we can segment a test corpus by first splitting it into individual characters (replacing all whitespaces with the "_" end of word character). Then we replace all occurances of adjacent tokens from the first merged pair with the merged pair token. Then we repeat this for the second pair and so on.


In [12]:
""" 
    Define some helper functions that will be used by the token learner
"""

# splits corpus into words, returns a dictionary of word frequencies
def init_vocab_words(corpus, eow_token="_"):
    vocab = list(set(list("".join(corpus.split())))) + [eow_token]
    # split corpus across whitespaces and make all characters lower case
    words = corpus.lower().strip().split()
    # get word frequencies
    word_freq = collections.defaultdict(int)
    for word in words:
        # store the word key as a string containing tokens separated by whitespace
        word_key = " ".join(list(word+eow_token))
        word_freq[word_key] += 1

    return vocab, word_freq


# find all unique pairs of adjacent tokens and their counts
def get_pairs(word_freq):
    pairs = collections.Counter()
    for word, freq in word_freq.items():
        tokens = word.split()
        for pair in zip(tokens[:-1], tokens[1:]):
            pairs[pair] += freq 
    return pairs


# merge a pair of tokens (token learner)
def merge_pair_learner(pair, word_freq):
    word_freq_new = collections.defaultdict(int)
    pattern = " ".join(pair)
    pattern_joined = "".join(pair) 
    if not pair[1].endswith("_"):
        pattern = pattern + " "
        pattern_joined = pattern_joined + " "

    for word, freq in word_freq.items():
        # merge all occurances of the pair in every word
        word_new = re.sub(pattern, pattern_joined, word)
        word_freq_new[word_new] = freq
    return word_freq_new

"""
    BPE token learner 
"""
def bpe_learn(corpus, k=100):
    
    # get inital vocab, word and token pair frequencies
    vocab, word_freq = init_vocab_words(corpus)
    #print(f"word_freq: {word_freq}")

    # performs mergers to learn the vocabulary
    merged_pairs = []
    for _ in range(k):
        # get counts of all adjacent token pairs
        pairs = get_pairs(word_freq)
        if not pairs:
            break
        # get most frequent pair
        most_freq_pair = pairs.most_common(1)[0][0]
        #print(f"best pair: {most_freq_pair}")

        # apply merger
        word_freq = merge_pair_learner(most_freq_pair, word_freq)
        #print(f"word_freq after merge: {word_freq}")

        # add merged token to vocab
        vocab.append("".join(most_freq_pair))
        merged_pairs.append(most_freq_pair)

    # precompute tokenized words
    word_tokens = collections.defaultdict(list)
    corpus_words = corpus.lower().strip().split()
    for word in corpus_words:
        word_tokens[word] = tokenize_word(word, vocab, merged_pairs)   

    #print(f"merged pairs: {merged_pairs}")    
    #print(f"final vocab: {vocab}")    
    #print(f"precomputed tokenizations: {word_tokens}")

    return vocab, merged_pairs, word_tokens    
    

# merge a pair of tokens (token segmenter)
def merge_pair_segmenter(pair, corpus_tokens):
    pattern = " ".join(pair)
    pattern_joined = "".join(pair) 
    if not pair[1].endswith("_"):
        pattern = pattern + " "
        pattern_joined = pattern_joined + " "
    # merge all occurances of the pair in every word
    corpus_tokens_new = re.sub(pattern, pattern_joined, corpus_tokens)
    return corpus_tokens_new


# encode a word into a list of BPE tokens
def tokenize_word(word, vocab, pairs):
    # split word into individual characters separated by white spaces, also insert eow token
    word_tokens = " ".join(list(word)) + " _"
    # now replace all occurances of pairs with the merged pair
    for pair in pairs:
        word_tokens =  merge_pair_segmenter(pair, word_tokens)
        if len(word_tokens.split()) == 1:
            break
    
    word_tokens = word_tokens.split()
    return word_tokens

""" 
    BPE token segmenter
"""    
def bpe_segment(test_corpus, vocab, pairs, precomputed_word_tokens):
    # split corpus into words
    corpus_words = test_corpus.split()
    # now replace all occurances of pairs with the merged pair
    #print(f"Corpus words: \n{corpus_words}")
    corpus_tokens = []
    for word in corpus_words:
        if word in precomputed_word_tokens:
            word_tokens = precomputed_word_tokens[word]
        else:    
            word_tokens = tokenize_word(word, vocab, pairs)
        #print(f"{word} ---> {word_tokens}")
        corpus_tokens.extend(word_tokens)    
    return corpus_tokens    

We can test our implemntation on a small example corpus

In [13]:
example_corpus = "low "*5 + "lowest "*2 + "newer "*6 + "wider "*3 + "new "*2
print(example_corpus)

# learn the vocabulary
vocab, merged_pairs, word_tokens  = bpe_learn(example_corpus)

# encode a test sentence
s = "Yes, they said we need to lower it."
s_encoded = bpe_segment(s, vocab, merged_pairs, word_tokens)
print(s_encoded)

low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new 
['Y', 'e', 's', ',', '_', 't', 'h', 'e', 'y', '_', 's', 'a', 'i', 'd', '_', 'w', 'e', '_', 'ne', 'e', 'd', '_', 't', 'o', '_', 'low', 'er_', 'i', 't', '.', '_']


#### Now let's organize the code into a tokenizer class

In [14]:
from tqdm import tqdm

In [26]:
class BPE():

    def __init__(self, max_vocab_size=100):
        self.max_vocab_size = max_vocab_size
    
    # splits corpus into words, returns a dictionary of word frequencies
    def init_vocab_words(self, corpus, eow_token="_"):
        vocab = list(set(list("".join(corpus.split())))) + [eow_token]
        # split corpus across whitespaces and make all characters lower case
        words = corpus.lower().strip().split()
        print(f"Number of words in corpus: {len(words)}")
        # get word frequencies
        word_freq = collections.defaultdict(int)
        for word in words:
            # store the word key as a string containing tokens separated by whitespace
            word_key = " ".join(list(word+eow_token))
            word_freq[word_key] += 1

        return vocab, word_freq

    # find all unique pairs of adjacent tokens and their counts
    def get_pairs(self, word_freq):
        pairs = collections.Counter()
        for word, freq in word_freq.items():
            tokens = word.split()
            for pair in zip(tokens[:-1], tokens[1:]):
                pairs[pair] += freq 
        return pairs


    # merge a pair of tokens (token learner)
    def merge_pair_learner(self, pair, word_freq, iter):
        word_freq_new = collections.defaultdict(int)
        pattern = " ".join(pair)
        pattern_joined = "".join(pair) 
        if not pair[1].endswith("_"):
            pattern = pattern + " " # match whitespace at the end
            pattern_joined = pattern_joined + " "  
        pattern = re.escape(pattern) # special characters need to be escaped   
        for word, freq in word_freq.items():
            # merge all occurances of the pair in every word
            word_new = re.sub(pattern, pattern_joined, word)
            word_freq_new[word_new] = freq
        return word_freq_new

    def learn(self, corpus):
        # get inital vocab, word and token pair frequencies
        vocab, word_freq = self.init_vocab_words(corpus)
        # performs mergers to learn the vocabulary
        merged_pairs = []        
        pbar = tqdm(total=self.max_vocab_size, desc="Building vocab. Num tokens added --> ")
        for _ in range(self.max_vocab_size):
            # get counts of all adjacent token pairs
            pairs = self.get_pairs(word_freq)
            if not pairs:
                break
            # get most frequent pair
            most_freq_pair = pairs.most_common(1)[0][0]
            # apply merger
            word_freq = self.merge_pair_learner(most_freq_pair, word_freq, _)
            # add merged token to vocab
            vocab.append("".join(most_freq_pair))
            merged_pairs.append(most_freq_pair)
            pbar.update(1)
            
        self.vocab = vocab
        self.merged_pairs = merged_pairs

        # precompute tokenized words
        word_tokens = collections.defaultdict(list)
        corpus_words = set(corpus.lower().strip().split())
        pbar = tqdm(total=len(corpus_words), desc="Precomputing word tokenizations --> ")
        for word in corpus_words:
            word_tokens[word] = self.tokenize_word(word)   
            pbar.update(1)
        self.word_tokens = word_tokens


    # merge a pair of tokens (token segmenter)
    def merge_pair_segmenter(self, pair, tokens):
        pattern = " ".join(pair)
        pattern_joined = "".join(pair) 
        if not pair[1].endswith("_"):
            pattern = pattern + " "
            pattern_joined = pattern_joined + " "
        pattern = re.escape(pattern)        
        # merge all occurances of the pair in every word
        tokens_new = re.sub(pattern, pattern_joined, tokens)
        return tokens_new

    # encode a word into a list of BPE tokens
    def tokenize_word(self, word):
        # split word into individual characters separated by white spaces, also insert eow token
        word_tokens = " ".join(list(word)) + " _"
        # now replace all occurances of pairs with the merged pair
        for pair in self.merged_pairs:
            word_tokens =  self.merge_pair_segmenter(pair, word_tokens)
            if len(word_tokens.split()) == 1:
                break
        
        word_tokens = word_tokens.split()
        return word_tokens

    # encode a test corpus
    def encode(self, test_corpus):
        # split corpus into words
        corpus_words = test_corpus.split()
        # now replace all occurances of pairs with the merged pair
        corpus_tokens = []
        for word in corpus_words:
            if word in self.word_tokens:
                tokens = self.word_tokens[word]
            else:    
                tokens = self.tokenize_word(word)
            corpus_tokens.extend(tokens)    
        return corpus_tokens    


In [27]:
example_corpus = "low "*5 + "lowest "*2 + "newer "*6 + "wider "*3 + "new "*2
print(example_corpus)

bpe_encoder = BPE()
bpe_encoder.learn(example_corpus)

# encode a test sentence
s = "Yes, they said we need to lower it."
s_encoded = bpe_encoder.encode(s)
print(s_encoded)

low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new 
Number of words in corpus: 18


Precomputing word tokenizations --> : 100%|██████████| 5/5 [00:00<00:00, 9873.60it/s]

['Y', 'e', 's', ',', '_', 't', 'h', 'e', 'y', '_', 's', 'a', 'i', 'd', '_', 'w', 'e', '_', 'ne', 'e', 'd', '_', 't', 'o', '_', 'low', 'er_', 'i', 't', '.', '_']


#### We can test on a bigger corpus from NLTK.

In [17]:
import nltk
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

# raw text from "Emma" by Jane Austen
emma = nltk.corpus.gutenberg.words('austen-emma.txt')

[nltk_data] Downloading package gutenberg to /home/tanzid/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [28]:
bpe_encoder = BPE(max_vocab_size=2000)
bpe_encoder.learn(" ".join(emma))

Number of words in corpus: 192427


Building vocab. Num tokens added --> : 100%|██████████| 2000/2000 [00:14<00:00, 135.11it/s]














































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [29]:
# encode a test sentence
s = "Yes, they said we need to lower it."
s_encoded = bpe_encoder.encode(s)
print(s_encoded)

['Y', 'es', ',_', 'they_', 'said_', 'we_', 'need_', 'to_', 'low', 'er_', 'it', '._']


In [35]:
# decoding token sequence back to a sentence is easy, just merge tokens and put a space after every token that ends with the "_" character
def decode(tokens):
    decoded_sentence = ""
    for token in tokens:
        if token.endswith("_"):
            decoded_sentence = decoded_sentence + token.strip("_") + " "
        else:
            decoded_sentence = decoded_sentence + token    
    return decoded_sentence.rstrip()        

In [36]:
decoded_sentence = decode(s_encoded)
decoded_sentence

'Yes, they said we need to lower it.'